In [2]:
import pandas as pd
import os
import numpy as np

In [5]:
# PATH = "./wandb_classification_mlp_tp_0505.csv"
# dataset = pd.read_csv(PATH)
PATH = "./wandb_classification_mlp_ccl_0505_2.csv"
dataset = pd.read_csv(PATH)
# dataset = pd.concat([dataset, dataset2])

# del dataset["Created"], dataset["Runtime"], dataset["GradC"], dataset["act_B"], dataset["act_F"], dataset["epochs"], dataset["loss_scale_ssl"]
# del dataset["User"], dataset["Group"], dataset["Job Type"], dataset["Tags"], dataset["Sweep"], dataset["Description"], dataset["GPU Count"], dataset["best_epoch"], dataset["num_chn"], dataset["tmax"], dataset["warmup"], dataset["eta_min"]
# if one of the above colum is in the dataset, delete it

for col_name in dataset.columns:
    if col_name in ["Created", "Runtime", "GradC", "act_B", "act_F", "epochs", "loss_scale_ssl", "User", "Group", "Job Type", "Tags", "Sweep", "Description", "GPU Count", "best_epoch", "num_chn", "tmax", "warmup", "eta_min"]:
        del dataset[col_name]

print("Size of dataset: ", len(dataset))
dataset = dataset[dataset["State"]=="finished"]
# dataset = dataset[dataset["architecture"]=="cnn_pool"]
# dataset = dataset[dataset["batchsize"]==64]
# dataset = dataset[dataset["fw_bn"]==0]
print("Size of dataset: ", len(dataset))

Size of dataset:  387
Size of dataset:  381


In [6]:
for task in ['MNIST', 'FashionMNIST', "CIFAR10", "CIFAR100"]:
    for method in ["CCL"]:
        ds = dataset[dataset["dataset"] == task]
        ds = ds[ds["method"] == method]
        # if task == "CIFAR10" and method == "CCL":
        #     ds = ds[ds["lr_F"] != 1.2]
        ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "bw_bn", "loss_scale_C"]).mean()
        ds_count = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "bw_bn", "loss_scale_C"]).count()
        ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "bw_bn", "loss_scale_C"]).std()
        ds_mean = ds_mean[ds_count["test_acc"] > 2]
        ds_std = ds_std[ds_count["test_acc"] > 2]
        try:
            row_with_highest_val_acc = ds_mean["val_acc"].idxmax()
        except:
            continue
        print("dataset: ", task, " method: ", method)
        print(row_with_highest_val_acc)
        print(f"mean: {ds_mean.loc[row_with_highest_val_acc]['test_acc']*100:.2f} std: {ds_std.loc[row_with_highest_val_acc]['test_acc']*100:.2f} count: {ds_count.loc[row_with_highest_val_acc]['test_acc']}")

        # assert 0 == 1

dataset:  MNIST  method:  CCL
(0.5, 1.5, 1.5, 0, 0, 0.0, 0.0, 0, 0.5)
mean: 98.13 std: 0.10 count: 5
dataset:  FashionMNIST  method:  CCL
(0.5, 1.5, 1.5, 0, 0, 0.0, 0.0, 0, 0.5)
mean: 88.58 std: 0.29 count: 5
dataset:  CIFAR10  method:  CCL
(0.5, 1.0, 1.0, 0, 0, 0.0, 0.0, 0, 0.5)
mean: 52.73 std: 0.59 count: 6
dataset:  CIFAR100  method:  CCL
(0.3, 2.0, 2.0, 0, 0, 0.0, 0.0, 0, 1.0)
mean: 20.85 std: 0.33 count: 5


/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_39397/1866989362.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "bw_bn", "loss_scale_C"]).mean()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_39397/1866989362.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "bw_bn", "loss_scale_C"]).std()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_39397/1866989362.py:7: FutureWarning: The default value of num

In [24]:
ds_mean

val_acc  \
grad_clip_F lr_F lr_B mmt_F mmt_B wd_F   wd_B   wd_B   bw_bn loss_scale_C             
0.5         0.5  0.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.977333   
                                  0.0001 0.0001 0.0001 0     0.5           0.969500   
            1.0  1.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.980000   
                                  0.0001 0.0001 0.0001 0     0.5           0.969111   
            1.5  1.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.981056   
                                  0.0001 0.0001 0.0001 0     0.5           0.969111   
            2.0  2.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.979500   
                                  0.0001 0.0001 0.0001 0     0.5           0.967833   
1.0         0.5  0.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.976778   
                                  0.0001 0.0001 0.0001 0     0.5           0.969111   
            1.0  1.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.979667   
                                  0.0001 0.0001 0.0001 0     0.5           0.969611   
            1.5  1.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.979056   
                                  0.0001 0.0001 0.0001 0     0.5           0.968944   
            2.0  2.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.977611   
                                  0.0001 0.0001 0.0001 0     0.5           0.968611   

                                                                           test_acc  \
grad_clip_F lr_F lr_B mmt_F mmt_B wd_F   wd_B   wd_B   bw_bn loss_scale_C             
0.5         0.5  0.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.979933   
                                  0.0001 0.0001 0.0001 0     0.5           0.973167   
            1.0  1.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.982200   
                                  0.0001 0.0001 0.0001 0     0.5           0.974067   
            1.5  1.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.981300   
                                  0.0001 0.0001 0.0001 0     0.5           0.973800   
            2.0  2.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.981167   
                                  0.0001 0.0001 0.0001 0     0.5           0.973433   
1.0         0.5  0.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.979300   
                                  0.0001 0.0001 0.0001 0     0.5           0.972767   
            1.0  1.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.980700   
                                  0.0001 0.0001 0.0001 0     0.5           0.974267   
            1.5  1.5  0     0     0.0000 0.0000 0.0000 0     0.5           0.981833   
                                  0.0001 0.0001 0.0001 0     0.5           0.973867   
            2.0  2.0  0     0     0.0000 0.0000 0.0000 0     0.5           0.980200   
                                  0.0001 0.0001 0.0001 0     0.5           0.973867   

                                                                           batchsize  \
grad_clip_F lr_F lr_B mmt_F mmt_B wd_F   wd_B   wd_B   bw_bn loss_scale_C              
0.5         0.5  0.5  0     0     0.0000 0.0000 0.0000 0     0.5               256.0   
                                  0.0001 0.0001 0.0001 0     0.5               256.0   
            1.0  1.0  0     0     0.0000 0.0000 0.0000 0     0.5               256.0   
                                  0.0001 0.0001 0.0001 0     0.5               256.0   
            1.5  1.5  0     0     0.0000 0.0000 0.0000 0     0.5               256.0   
                                  0.0001 0.0001 0.0001 0     0.5               256.0   
            2.0  2.0  0     0     0.0000 0.0000 0.0000 0     0.5               256.0   
                                  0.0001 0.0001 0.0001 0     0.5               256.0   
1.0         0.5  0.5  0     0     0.0000 0.0000 0.0000 0     0